# Segmenting and Clustering Neighborhoods in Toronto
### Part 1

In [27]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [28]:
postal_file = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(postal_file, 'lxml')

In [29]:
columns = ['PostalCode', 'Borough', 'Neighborhood']
df_tor = pd.DataFrame(columns=columns)
df_tor

,PostalCode,Borough,Neighborhood


In [30]:
def addRowToData(post, bor, neigh, data):
    if bor == 'Not assigned':
        return
    
    if neigh == 'Not assigned':
        neigh = bor
    
    data.append({'PostalCode':post, 'Borough':bor, 'Neighborhood':neigh})

In [31]:
rows = soup.find('tbody').find_all('tr')[1:]
data = []
dupCodeData = []
setOfCodes = set()

for row in rows:
    row_content = row.find_all('td')
    post = row_content[0].text
    bor = row_content[1].text
    neigh = row_content[2].text.split('\n')[0]
    
    if post in setOfCodes:
        addRowToData(post, bor, neigh, dupCodeData)
    else:
        setOfCodes.add(post)
        addRowToData(post, bor, neigh, data)
        
df_tor = pd.DataFrame(data)
df_dupes = pd.DataFrame(dupCodeData)

df_tor = df_tor[['PostalCode', 'Borough', 'Neighborhood']]

In [32]:

df_tor.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,Rouge
7,M3B,North York,Don Mills North
8,M4B,East York,Woodbine Gardens
9,M5B,Downtown Toronto,Ryerson


In [33]:
for index, row in df_dupes.iterrows():
    newNeighborhoodString = '{}, {}'.format(df_tor.loc[df_tor['PostalCode']==row['PostalCode'],:].values[0][2], row['Neighborhood'])
    df_tor.loc[df_tor['PostalCode']==row['PostalCode'], 'Neighborhood'] = newNeighborhoodString
    
df_tor.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


In [34]:
df_tor.shape

(103, 3)